In [10]:
import os
from openai import OpenAI
import markdown
import base64

In [12]:
# 读取api key的txt文件
with open(r"D:\Desktop\其他事务\gpt_api_key.txt", 'r') as f:
    API_KEY = f.read() 

In [13]:
model_name = "gpt-4o-mini-2024-07-18"
# model_name = "gpt-4o-2024-08-06"
if model_name == "gpt-4o-mini-2024-07-18":
    model_abbr = "mini"
else:
    model_abbr = "4o"
client = OpenAI(
    api_key=API_KEY
)

In [14]:
qs_file_path = r"D:\Desktop\intelligent_customer_service\q_converted_file.jsonl"
example_q_a_path = r"D:\Desktop\a_man.jsonl"

In [15]:
# 读取jsonl文件
import json
item = {}
with open(example_q_a_path, 'r', encoding='utf-8') as file:
    for line in file:
        item = json.loads(line.strip())
print(item)

{'促销和优惠咨询_活动': {'这个活动是只有618节才有吗': '是的呢，不过亲亲，您可以先收藏宝贝，订阅提醒，到时候活动开始，您就可以享受优惠啦~'}}


In [16]:
merged_q_path = r"D:\Desktop\intelligent_customer_service\merged_file.jsonl"
q_str_dict_list = []
with open(merged_q_path, 'r', encoding='utf-8') as file:
    for line in file:
        item = json.loads(line.strip())
        q_str_dict = {}
        # 创建新的字典
        for key, value in item.items():
            q_str_dict[key] = value[1].split('\n\n')[1]
        q_str_dict_list.append(q_str_dict)
print(q_str_dict_list)

[{'产品信息查询_规格大小': '1. 这个电视机是多大的呢？\n2. 这款手机的屏幕尺寸是多少？\n3. 这个沙发的尺寸是多少？\n4. 这台冰箱的容量有多大？\n5. 这款笔记本电脑的屏幕大小是多少？\n6. 这个床的尺寸是多大？\n7. 这款手表的表盘直径是多少？\n8. 这个包包的尺寸是多少？\n9. 这台洗衣机的容量是多少公斤？\n10. 这个餐桌的尺寸是多少？\n11. 这款耳机的耳罩直径有多大？\n12. 这个鞋子的尺码是多大？\n13. 这款相机的镜头直径是多少？\n14. 这个花盆的直径有多大？\n15. 这款游戏机的尺寸是多少？\n16. 这个音响的大小是多少？\n17. 这款手提箱的尺寸是多少？\n18. 这个瑜伽垫的长度和宽度是多少？\n19. 这款电视柜的尺寸是多少？\n20. 这个水壶的容量是多少升？\n21. 这款运动鞋的尺码是多大？\n22. 这个灯具的高度和直径是多少？\n23. 这款电脑显示器的屏幕尺寸是多少？\n24. 这个冰淇淋机的容量有多大？\n25. 这款婴儿车的折叠尺寸是多少？\n26. 这个健身器材的尺寸是多少？\n27. 这款耳机的线长是多少？\n28. 这个行李箱的尺寸是多少？\n29. 这款手提电脑的厚度是多少？\n30. 这个沙发床的尺寸是多少？\n31. 这款相框的尺寸是多少？\n32. 这个水族箱的容量是多少升？\n33. 这款咖啡机的尺寸是多少？\n34. 这个烤箱的内部容量有多大？\n35. 这款电饭煲的容量是多少升？\n36. 这个书架的高度和宽度是多少？\n37. 这款游戏手柄的尺寸是多少？\n38. 这个耳机的重量是多少克？\n39. 这款电动牙刷的长度是多少？\n40. 这个行李箱的重量是多少？\n41. 这款手提包的尺寸是多少？\n42. 这个健身球的直径有多大？\n43. 这款电风扇的高度是多少？\n44. 这个烤肉架的尺寸是多少？\n45. 这款水壶的高度和直径是多少？\n46. 这个笔记本的尺寸是多少？\n47. 这款手机壳的尺寸是多少？\n48. 这个电动滑板车的尺寸是多少？\n49. 这款耳机的耳塞大小有多大？\n50. 这个洗碗机的容量是多少？\n51. 这款电动剃须刀的尺寸是多少？\n52. 这个手提箱的容量是多少升？\n53. 这款相机的尺寸是多少？\n54. 这个花瓶的高度

In [17]:
total_num_gen = 1500
q_aspect_index = 0
begin, end = 6, 7
line_count = 12
per_aspect_num_gen = total_num_gen/line_count
q_a_lines = []
with open(example_q_a_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if begin <= i < end:
            # line 是二层字典
            line_dict = json.loads(line)  # 如果 line 是 JSON 字符串，先解析成字典
            # 提取外层键
            outer_key = next(iter(line_dict))
            # 提取内层字典
            inner_dict = line_dict[outer_key]
            # 提取内层键和值
            inner_key = next(iter(inner_dict))
            inner_value = inner_dict[inner_key]
            # 现在我们可以使用这些变量
            q_aspect = outer_key
            exp_q = inner_key
            exp_a = inner_value
            broad, specific = q_aspect.split('_')
            system_prompt = "你是一位温柔、可爱、高情商的客服，对顾客的问题总会耐心解答，给出让顾客满意的回答"
            user_prompt = \
            f'''作为一位始终保持温柔、可爱、高情商的客服，你要回复不同顾客提出的关于{specific}的{broad}方面的{per_aspect_num_gen}个问题。
            例如：
            0. {exp_q} 回复: {exp_a}
            下面是不同顾客提出的{per_aspect_num_gen}个问题，请模仿上述示例的语气风格依次给出回复：
            {q_str_dict_list[i][q_aspect]}
            回复格式：\n序号. 回复: 回复内容
            '''
            completion = client.chat.completions.create(
                model = model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                frequency_penalty=0,
                presence_penalty= -0.5,
                max_tokens=4096,
                temperature=0,
                top_p=0.2,
                n=1,
                seed=42
                )
            reply = completion.choices[0].message.content
            # 将reply append到list变量line[q_aspect]中
            line_dict[q_aspect] = reply
            q_a_lines.append(line_dict)
        elif i >= end:
            break

In [18]:
OUTPUT_FILE_PATH = \
    fr"D:\Desktop\intelligent_customer_service\\{model_abbr}_q_a_{begin}_{end}.jsonl" 
# 将list变量reply转换为jsonl文件
with open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as outfile:
    for item in q_a_lines:
        json.dump(item, outfile, ensure_ascii=False)
        outfile.write('\n')